If we assume that any meaningful airport business (bag check, security, walking to the gate, etc.) takes a time $T$ hours, then we can define the cost of arriving $\tau$ hours before takeoff as
$$C(\tau, t) = \begin{cases}
  \int_{0}^{\tau-T}c_{w}(t')dt' & T < \tau\\
  C_{M}  & T \geq \tau
  \end{cases}
$$
Where $c_{w}$ is the cost per unit time of waiting at the airport, and $C_M$ is the total cost of missing your flight. To keep things simple, let's assume that your cost per unit time is constant

$$c_{w}(t')\ =\ c_{w}$$

and model $T$ as a random variable over which we can marginalize to get the expected cost of arriving at the airport $\tau$ hours before your flight

$$C(\tau) = \mathbb{E}\big{[}C(\tau, T)\big{]}\ =\ \int_{0}^{\infty}C(\tau, t)p_{T}(t)dt\ .$$

Breaking up this integral using the formulas for $C(\tau, t)$ and $c_{w}$ above, we get

$$C(\tau) = c_{w}\int_{0}^{\tau}(\tau - t)p_{T}(t)dt + C_{m}\int_{T}^{\infty}p_{T}(t)dt\ .$$

If we let $P_{T}$ represent the cumulative density function for $T$, we can turn the second term above into

$$C_{m}\int_{T}^{\infty}p_{T}(t)dt\ =\ C_{m}\big{[}1 - P_{T}(\tau)\big{]}\ .$$

The first term can be broken up into two parts. The first will be

$$c_{w}\int_{0}^{\tau}\tau\ p_{T}(t)dt\ =\ c_{w}\ \tau\ P_{T}(\tau)\ .$$

For the second, if we let $g(\tau)$ be the partial expectation of $T$

$$g(\tau)\ =\ \int_{\tau}^{\infty}t\ p_{T}(t)dt ,$$

then this becomes

$$-c_{w}\int_{0}^{\tau}t\ p_{T}(t)dt\ =\ -c_{w}\big{[}\mathbb{E}[T]\ -\ g(\tau)\big{]}\ .$$

So we can rewrite everything as

$$C(\tau)\ =\ c_{w}\ \tau\ P_{T}(\tau)\ -\ c_{w}\big{[}\mathbb{E}[T]\ -\ g(\tau)\big{]}\ +\ C_{m}\big{[}1 - P_{T}(\tau)\big{]}\ ,$$

which with a little rearrangement can be expressed as

$$C(\tau)\ =\ C_{M}\ +\ P_{T}(\tau)\big{[}c_{w}\tau\ -\ C_{M}\big{]}-\ c_{w}\big{[}\mathbb{E}[T]\ -\ g(\tau)\big{]}\ .$$

This has all been done without specifying a distribution for $T$. If we assume that $T$ follows a log-normal distribution, with $\mu$ and $\sigma$ representing the mean and standard deviation of $T$'s natural logarithm, then we can plug in values for $\mathbb{E}[T]$, $P_{T}(\tau)$, and $g(\tau)$

$$\mathbb{E}[T]\ =\ e^{\mu + \frac{\sigma^2}{2}}$$
$$P_{T}(\tau)\ =\ \frac{1}{2}\text{erfc} (\frac{\mu-\ln{\tau}}{\sigma \sqrt{2}})$$
$$g(\tau)\ =\ \frac{1}{2} \mathbb{E}[T] \big{[}1 + \text{erf}(\frac{\mu+\sigma^2-\ln(\tau)}{\sigma\sqrt{2}})\big{]}\ .$$

Plugging these in above gives us, after some rearranging,

$$C(\tau)\ =\ C_{M}\ +\ \frac{1}{2}\text{erfc} (\frac{\mu-\ln{\tau}}{\sigma \sqrt{2}})[c_{w}\tau-C_{M}]\ -\ \frac{c_{w}}{2} e^{\mu+\frac{\sigma^2}{2}}\text{erfc}(\frac{\mu+\sigma^2-\ln{\tau}}{\sigma\sqrt{2}})\ .$$

Since our cost units are imaginary, I'll downscale everything by $C_{M}$ to put everything in terms of the relative cost of $c_w$ and $C_M$

$$C(\tau)\ =\ 1\ +\ \frac{1}{2}\text{erfc} (\frac{\mu-\ln{\tau}}{\sigma \sqrt{2}})\big{[}\frac{c_{w}}{C_{M}}\tau-1\big{]}\ -\ \frac{c_{w}}{2C_{M}} e^{\mu+\frac{\sigma^2}{2}}\text{erfc}(\frac{\mu+\sigma^2-\ln{\tau}}{\sigma\sqrt{2}})\ .$$

Finally, we can use <a href="https://www.tsa.gov/news/releases/2019/01/19/tsa-statement-checkpoint-operations-january-19">this very rough data</a> put out by the TSA to plug in some numbers and plot the cost curve for a few different cost ratios.

In [1]:
import numpy as np
from scipy.special import erfc, erfinv

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import YlGnBu4
output_notebook()

# account for boarding time in hours
boarding_buffer = 0.5

# invert the cumulative distribution formula to get a
# system of equations for mu and sigma
wait_times = [0.25, 0.5]
log_wait_times = [np.log(time) for time in wait_times]

wait_probabilities = [0.912, .997]
sigma_coeffs = [erfinv(p)*2 for p in wait_probabilities]
mu_coeffs = [1, 1]
coeff_matrix = np.array([mu_coeffs, sigma_coeffs]).T

mu, sigma = np.linalg.solve(coeff_matrix, log_wait_times)

# return to time space to get a sense for how our variables are distributed
log_mu = np.exp(mu + sigma**2/2) + boarding_buffer
log_var = (np.exp(sigma**2) - 1)*log_mu**2
print("Mean wait time: {} minutes".format(int(log_mu*60)))
print("Std: {} minutes".format(int((log_var**0.5)*60)))

# account for our boarding buffer in mu
mu = np.log(np.exp(mu) + boarding_buffer)

# build the various terms above, excluding those dependent on the ratio
tau_start, tau_end = boarding_buffer*0.5, boarding_buffer*2.5
taus = np.linspace(tau_start, tau_end, 100)
a = 1
b = 0.5*erfc((mu-np.log(taus))/(sigma*np.sqrt(2)))
c = 0.5*np.exp(mu+0.5*sigma**2)*erfc((mu+sigma**2-np.log(taus))/(sigma*np.sqrt(2)))

# use bokeh to make our plot pretty
ds = ColumnDataSource({'tau': taus*60})
p = figure(
    plot_height=500,
    plot_width=800,
    x_range=(tau_start*60, tau_end*60),
    tools="",
    title="Expected Cost of Airport Arrival",
    background_fill_color="#eff1f4",
    background_fill_alpha=0.8
)

x_mins, y_mins = [], []
missed_flight_relative_costs = [1, 2, 4, 10]
for n, relative_cost in enumerate(missed_flight_relative_costs):
    ratio = 1.0/relative_cost
    C = a + b*(ratio*taus-1) - c*ratio
    ds.data[str(relative_cost)] = C
    p.line(
        'tau',
        str(relative_cost),
        legend="{}x".format(relative_cost),
        source=ds,
        line_alpha=0.8,
        line_color=YlGnBu4[n],
        line_width=3,
        name=str(relative_cost))

    # keep track of our cost minimization points
    y_mins.append(np.min(C))
    x_mins.append(taus[np.argmin(C)]*60)

p.x(x_mins, y_mins, size=5, fill_color='black', line_color='black', legend='Minimum')

# do some figure formatting
p.xaxis.axis_label = "Arrival Time Before Takeoff in Minutes"
p.yaxis.axis_label = "Expected Total Cost, Relative Units"
p.yaxis.ticker = []

p.grid.grid_line_color = "#ffffff"
p.grid.grid_line_width = 2

p.legend.title = "Relative cost of missing flight compared to an hour at the airport"
p.legend.orientation = 'horizontal'
p.legend.glyph_width = 40

# add a hover tool
tooltips = [("Arrive", "@tau minutes early")]
tooltips.extend(
    [("{}x".format(ratio), "@{}".format(ratio))
     for ratio in missed_flight_relative_costs])
p.add_tools(HoverTool(
  mode='vline',
  tooltips=tooltips,
  renderers=p.renderers[-5:-4]
))
show(p)

Loading BokehJS ...

Mean wait time: 36 minutes
Std: 14 minutes


So it largely comes down to a matter of your personal choice of cost function. If spending an hour at the airport is as unthinkable to you as missing a flight, then feel free to arrive 45 minutes early. If you see waiting at the gate as a pleasant opportunity to read or answer emails, or just can't afford to miss your friend's wedding, then maybe give yourself an hour fifteen or so.